In [20]:
import requests as rq
import bs4 as bs
import re
import time

In [15]:
url = f"https://www.atlasobscura.com/places?page=202&sort=likes_count"
raw = rq.get(url)
soup = bs.BeautifulSoup(raw.content)

#ciao

In [18]:
raw.status_code == 200

True

In [9]:
len(soup.find_all("a",attrs={'data-place-id':re.compile(r"\d*")}))

0

In [ ]:
cities_url = []
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for page in range(1,401):
    try:
        url = f"https://www.atlasobscura.com/places?page={page}&sort=likes_count"
        raw = rq.get(url,headers=headers)
        if raw.status_code != 200:
            print("Sleeping")
            time.sleep(120)
            raw = rq.get(url,headers=headers)
        
        soup = bs.BeautifulSoup(raw.content)
        
        for d in soup.find_all("a",attrs={'data-place-id':re.compile(r"\d+")}):
            cities_url.append(d["href"])
            
    except Exception as e:
        print(page)
        print(e)
        print(raw.status_code)

In [26]:
len(cities_url)

7200

In [28]:
with open("links.txt","w") as f:
    for link in cities_url:
        complete_url = "https://www.atlasobscura.com"+link+"\n"
        f.write(complete_url)

In [19]:
url = f"https://www.atlasobscura.com/places?page=2&sort=likes_count"

In [20]:
raw = rq.get(url)
soup = bs.BeautifulSoup(raw.content)

In [22]:
with open("pretty.txt","w") as f:
    f.write(soup.prettify())

### Downloading the html files

_(nota per voi) il modulo qua sotto serve per generare degli headers random_

In [ ]:
!pip install fake-useragent

In [2]:
import os
from concurrent.futures import ThreadPoolExecutor, wait
from fake_useragent import UserAgent
ua = UserAgent()
import random

In [3]:
# create a list of fake headers
headers = [ua.random for _ in range(1000)]

In [4]:
with open("links.txt","r") as f:
    links = list(map(lambda x: x.rstrip("\n"), f.readlines()))

_salva tutti i file nella cartella "download", va modificato quel parametro per mettergli le pagine_

In [6]:
def parallel_html_downloader(links:list,directory="downloads"):
    if not os.path.exists(directory):
        os.makedirs(directory)
    """
    Parallelize html downloading by the means of multithreading
    """
    def downloader(url:str, directory="downloads"):
        try:
            raw = rq.get(url, headers={'User-Agent':random.choice(headers)})   # pick a random user agent for the request
            while raw.status_code != 200:
                #print(f"Key error: {raw.status_code}. Sleeping 180s...")    # wait 180 seconds before doing another get (too many requests)
                time.sleep(180)
                raw = rq.get(url, headers={'User-Agent':random.choice(headers)})

            with open(f"{directory}/{url.split('/')[-1]}.html","w") as f:    # save the file.html in the folder
                f.write(raw.text) 
        except Exception as e:
            print(e)
            
    with ThreadPoolExecutor() as executor:
        executor.map(downloader, links)    # parallelize the downloader function over list of links

In [7]:
%%time
parallel_html_downloader(links)

CPU times: user 1min 37s, sys: 7.84 s, total: 1min 45s
Wall time: 1h 56min 13s
